In [1]:
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')

# SLIDE (1) Первое обучение

Простое как пробка задание. Обучите классификатор [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) на входных данных с гиперпараметрами:
* max_depth=6
* min_samples_split=3
* min_samples_leaf=3
* n_estimators=100
* n_jobs=-1

И верните обученную модель.

Данные в X только численные, в y только 2 значения: 0 и 1. 

In [987]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

def fit_rf(X: np.ndarray, y:np.ndarray) ->  RandomForestClassifier:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (2) Первая классификация

По [ссылке](https://yadi.sk/d/xhbGrNuOjv6bjA) вы можете найти данные для бинарной классификации. $Y$ в этих данных выступает столбик `Outcome`, в качаестве $X$ - все остальное. 

Вам необходимо предсказать $y_{test}$ такой, что $accuracy > 0.75$ ([доля правильных ответов](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)). Вы можете делать что угодно получить результат:
* использовать любой классификатор с любыми гиперпараметрами
* как угодно изменять данные 

Вернуть в этом случае нужно не модель, а результат - одномерный массив данных $y_{pred}$ (предсказание $y_{test}$).

P.S. Можете узнать больше о данных по [ссылке](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Мы произвольным образом разбили данные в соотношении 4:1.

In [951]:
import numpy as np
import pandas as pd

def classification(X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray) -> np.ndarray:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (2) Переобучение

По [ссылке](https://yadi.sk/d/BtzKJULWGf0nng) вы можете найти данные для бинарной классификации. Вам на вход подается тренировачная и тестовая выборки из файла. Верните такую обученную модель, которая на тренировачной выборке дает $accuracy > 0.97$, а на тестовом $accuracy < 0.7$.

[$accuracy$](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) - доля правильных ответов.

In [657]:
def overfitting(X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (3) Мой KNN

Мы уже научились обучать разные задачи, теперь давайте напишем свой классификатор. Ваша задача реализовать свой простой KNN. Вам нужно реализовать 3 метода:
* `init` - начальная инициализация
* `fit` - обучение классификатора
* `predict` - предсказание для новых объектов
* `predict_proba` - предсказание вероятностей новых объектов

У нашего классификатора будет лишь один гиперпараметр - количество соседей $k$. Во избежании тонкостей: $k$ - нечетное.

На вход будет подаваться выборка объектов $X$, у которых ровно 2 числовых признака. $y$ - результат бинарной классификации $0$ или $1$.

Метрика ближайших элементов - Эвклидова.

Напоминание: $y$ - одномерный массив, $X$ - двумерный массив, по $0$-ой оси которой расположены объекты.

In [50]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier as KNN

class MyKNN():
    def __init__(self, k):
        self.k = k
        self.train_points = {}
        
    def fit(self, X_train: np.array, y_train:np.array): #обучаем классификатор
        n = X_train.shape[0]
        for i in range(n):
            point = tuple(X_train[i])
            res = y_train[i]
            self.train_points[point] = res
        return self

    def predict(self, X_test: np.array): #возвращаем значения
        result = []
        n = X_test.shape[0]
        for i in range(n):
            point = tuple(X_test[i])
            neighbors = self._get_neighbors(point)
            res = self._get_more_freq_res(neighbors)
            result.append(res)
        return np.array(result)           
                
    
    def predict_proba(self, X_test: np.array): #возвращаем значения
        result = []
        n = X_test.shape[0]
        for i in range(n):
            point = tuple(X_test[i])
            neighbors = self._get_neighbors(point)
            res = self._get_proba(neighbors)
            result.append(res)
        return np.array(result) 
    
    def _get_distance(self, point1, point2):
        return pow(point1[0] - point2[0], 2) + pow(point1[1] - point2[1], 2)
    
    def _get_neighbors(self, point):
        neighbors = {}
        max_dist = 0
        for train_point in self.train_points.keys():
            dist = self._get_distance(point, train_point)
            if len(neighbors) < self.k:
                neighbors[train_point] = dist
                max_dist = max(max_dist, dist)
            elif dist < max_dist:
                for p, d in neighbors.items():
                    if d == max_dist:
                        del neighbors[p]
                        break
                neighbors[train_point] = dist
                max_dist = max(neighbors.values())
        return neighbors.keys()
    
    def _get_more_freq_res(self, neighbors):
        zeros_count, ones_count = self._get_zeros_ones_count(neighbors)
        return 0 if zeros_count > ones_count else 1
    
    def _get_proba(self, neighbors):
        zeros, ones = self._get_zeros_ones_count(neighbors)
        return zeros / (zeros + ones), ones / (zeros + ones) 
        
    def _get_zeros_ones_count(self, neighbors):
        zeros_count = 0
        ones_count = 0
        for train_point in neighbors:
            res = self.train_points[train_point]
            if res == 0:
                zeros_count += 1 
            else:
                ones_count += 1
        return zeros_count, ones_count
        

'''
X_train = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_train = np.array([1, 1, 0, 0])
model = MyKNN(k=3).fit(X_train, y_train)
y_pred = model.predict(np.array([[0.5, 0.5], [ -0.5,  -0.5]]))
y_prob = model.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]]))
print(y_pred)
print(y_prob)
real = KNN(3).fit(X_train, y_train)
print(real.predict(np.array([[0.5, 0.5], [ -0.5,  -0.5]])))
print(real.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]])))
'''

X_train, y_train = make_classification(n_features=2, n_classes=2, n_samples=500, 
                                       n_informative=2, n_redundant=0, n_repeated=0)
X_test, _ = make_classification(n_features=2, n_classes=2, n_samples=500, 
                                n_informative=2, n_redundant=0, n_repeated=0)

k = 5
my_clf = MyKNN(k)
clf = KNN(k)
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)
my_predict = my_clf.predict(X_test)
real_predict = clf.predict(X_test)
bad = 0
for i in range(my_predict.shape[0]):
    if my_predict[i] != real_predict[i]:
        bad += 1
print('bad predicted ', bad)

my_predict_proba = my_clf.predict_proba(X_test)
print(my_predict_proba[:10])
real_predict_proba = clf.predict_proba(X_test)
print(real_predict_proba[:10])
bad = 0
for i in range(my_predict.shape[0]):
    if (my_predict_proba[i][0] != real_predict_proba[i][0]) or \
    (my_predict_proba[i][1] != real_predict_proba[i][1]):
        bad += 1
print('bad predicted ', bad)

bad predicted  0
[[0.  1. ]
 [0.2 0.8]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [1.  0. ]]
[[0.  1. ]
 [0.2 0.8]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [1.  0. ]]
bad predicted  0


# SLIDE (2) Моя Регрессия

Теперь вам предстоит реализовать свою простейшую линейную регрессию по функкционалу $MSE$.

Линейная регрессия выглядит следующим образом:
$$a(x) = w_1x + w_0$$

Необходимо найти такие $w_0$ и $w_1$ при которых минимизируется значение

$$MSE(X,Y) = \sum_{i=1}^{n}(a(x_i) - y_i)^2$$

Выведите формулы для $w_0$ и $w_1$ аналитически и реализуйте следующие методы класса 

* `init` - начальная инициализация
* `fit` - обучение классификатора
* `predict` - предсказание для новых объектов

После обучения у модели должен присутствовать атрибут `model.coef_` из которого можно получить коэффициенты регрессии в порядке: $w_1$, $w_0$.

Гиперпараметры отсутствуют

На вход будут подаваться два массива $X\in \mathbb{R}^{n}$ и $Y \in \mathbb{R}^{n}$

Метрика - Эвклидова.

Подсказка: для тестирования можете использовать реальную `LinearRegression`

In [100]:
from sklearn.linear_model import LinearRegression
import numpy as np

class LinReg():
    def __init__(self):
        self.coef_ = []

    def fit(self, X_train: np.array, y_train: np.array): #обучаем классификатор
        y_mean = np.mean(y_train)
        x_mean = np.mean(X_train)
        x_sqr_mean = np.mean(X_train * X_train)
        prod = []
        for i in range(y_train.shape[0]):
            prod.append(X_train[i] * y_train[i])
        prod_mean = np.mean(prod)
        b = (prod_mean - y_mean * x_mean) / (x_sqr_mean - x_mean * x_mean)
        a = y_mean - b * x_mean
        self.coef_ = [b, a]
        return self

    def predict(self, X_test: np.array): #возвращаем значения
        b, a = self.coef_[0], self.coef_[1]
        f = np.vectorize(lambda x: b * x + a)
        return f(X_test).flatten()
    
X_train = np.array([[1], [2], [4]])
y_train = np.array([0, 5, 8])

lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(np.array([[3], [4]]))
print(y_pred)
print(lr.coef_)

model = LinReg().fit(X_train, y_train)
y_pred = model.predict(np.array([[3], [4]]))
print(y_pred)
print(model.coef_)



[6.  8.5]
[2.5]
[6.  8.5]
[2.500000000000002, -1.5000000000000062]
